In [13]:
pip install tensorflow


Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-2.18.0-cp312-cp312-win_amd64.whl.metadata (3.3 kB)
  Using cached tensorflow_intel-2.18.0-cp312-cp312-win_amd64.whl.metadata (4.9 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-2.5.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached grpcio-1.68.0-cp312-cp312-win_amd64.whl.metadata (4.0 kB)
  Using cached tensorboard-2.18.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached keras-3.6.0-py3-none-any.whl.metadata (5.8 kB)
  Us

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


## ignoring warnings

In [1]:
import warnings
import pandas as pd
warnings.filterwarnings("ignore")

## Loading data

In [2]:
import torch
data = torch.load('../dataset/dataset/part_one_dataset/train_data/1_train_data.tar.pth')

## Analyzing data

In [3]:
print(data.keys())


dict_keys(['data', 'targets'])


In [4]:
print(data['data'].shape)
print(data['targets'].shape)

(2500, 32, 32, 3)
(2500,)


In [5]:
print(type(data['data']))

<class 'numpy.ndarray'>


In [6]:
print(data['data'][0].shape)

(32, 32, 3)


In [7]:
print(type(data))

<class 'dict'>


## Converting dict to df

In [8]:
print(type(data['targets']))
print(type(data['data']))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [9]:
print(data['targets'].shape)
print(data['data'].shape)

(2500,)
(2500, 32, 32, 3)


In [10]:
df = pd.DataFrame(data['targets'])
df.columns = ['targets']
print(df.head())

   targets
0        6
1        9
2        9
3        4
4        1


In [11]:
import seaborn as sns
import matplotlib.pyplot as plt

## Extracting features

In [12]:
import numpy as np
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model

# Example data (replace this with your actual data)
X_train = data['data']  # 2500 images, 32x32x3 shape
y_train = data['targets']   # 10 classes (0-9)

# Create a CNN model for feature extraction
input_layer = layers.Input(shape=(32, 32, 3))

# Convolutional Layer 1
x = layers.Conv2D(32, (3, 3), activation='relu')(input_layer)
x = layers.MaxPooling2D((2, 2))(x)

# Convolutional Layer 2
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)

# Flatten the output of the last convolutional layer
x = layers.Flatten()(x)

# Fully connected layers (optional for classification)
x = layers.Dense(128, activation='relu')(x)
output_layer = layers.Dense(10, activation='softmax')(x)

# Build the model
cnn_model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Perform a dummy pass to initialize the model and its weights
cnn_model.predict(np.random.rand(1, 32, 32, 3))  # A dummy pass to initialize the model

# Now, remove the classification layers and only output the features
# Access the last convolutional layer (before flattening) for feature extraction
feature_extractor = Model(inputs=cnn_model.input, outputs=cnn_model.layers[2].output)  # The second conv layer is at index 2

# Extract features from the CNN model (this will be a 2D array of shape (2500, num_features))
features = feature_extractor.predict(X_train)

print(f"Extracted Features Shape: {features.shape}")  # Should print the shape like (2500, num_features)


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
features_flat = features.reshape(features.shape[0], -1)

# Now, 'features_flat' contains the extracted features
print(features_flat.shape) 

## Dimensionality Reduction

In [ ]:
from sklearn.decomposition import PCA

# Assume features_flat is your flattened feature array from the pretrained model
# For example, features_flat.shape = (2500, 25088) for VGG16

# Initialize PCA to reduce the dimensionality
pca = PCA(n_components=0.95)  # Retain 95% of the variance
features_reduced = pca.fit_transform(features_flat)

print(features_reduced.shape)  

In [ ]:
print(type(features_reduced))

In [ ]:
print(features_reduced[0])

## Converting to Gaussian

In [ ]:
from sklearn.preprocessing import StandardScaler

# Initialize the StandardScaler
scaler = StandardScaler()

# Standardize the reduced feature set (features_reduced)
features_standardized = scaler.fit_transform(features_reduced)

# Now, 'features_standardized' is standardized and ready for use

print(features_standardized.shape)

In [ ]:
df_data = pd.DataFrame(features_standardized)

In [ ]:

print(df.head())

In [ ]:
df_concat = pd.concat([df_data,df],axis = 1)
print(df_concat.head())

In [ ]:
print((features_standardized))

In [ ]:
print(type(data['targets']))

## DO the lwp

In [ ]:


y = data['targets']
X = features_standardized



In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Reshape
from sklearn.metrics import accuracy_score

# Example: X is your input data (already transformed, standardized, and linearized)
# y is your labels (integers from 0 to 9 for 10 classes)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape data to fit LSTM input requirements (LSTM expects 3D input)
# Here, we reshape X_train and X_test from shape (num_samples, num_features) to (num_samples, timesteps, features)
# We can consider each feature as a timestep, with 1 feature per timestep (timesteps=1).

X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))  # (num_samples, timesteps=1, num_features)
X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))      # (num_samples, timesteps=1, num_features)

# Build the neural network model with an LSTM layer
model = Sequential()

# Input layer (since X has already been transformed, we start with the feature size directly)
model.add(LSTM(128, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]), activation='relu', return_sequences=False))  # LSTM layer

# Dense layers
model.add(Dense(128, activation='relu'))  # Second hidden layer
model.add(Dense(64, activation='softmax'))  # Output layer with 10 classes

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_reshaped, y_train, epochs=100, batch_size=32, validation_data=(X_test_reshaped, y_test))

# Evaluate the model on the test set
y_pred_prob = model.predict(X_test_reshaped)
y_pred = np.argmax(y_pred_prob, axis=1)  # Get the predicted class (the class with the highest probability)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')


### Analyzing Images

In [ ]:
from PIL import Image
import os
import tensorflow as tf

In [ ]:
for i in range(2500):
    img = Image.fromarray(data['data'][i])
    target_dir = f"./{data['targets'][i]}"
    os.makedirs(target_dir, exist_ok=True)
    img.save(f"{target_dir}/img{i}.png")

### PCA

In [ ]:
import tensorflow as tf
images = data['data']
# Enhanced CNN Model for feature extraction
model = tf.keras.Sequential([
    # First Convolutional Block
    tf.keras.layers.Conv2D(
        filters=4,  # Number of filters for learning low-level features
        kernel_size=(2, 2),  # Filter size
        strides=(1, 1),  # No down-sampling yet
        padding='same',  # Keeps output size same
        activation='relu',  # ReLU activation
        input_shape=(32, 32, 3)  # Input shape for RGB images
    ),
    tf.keras.layers.MaxPooling2D(
        pool_size=(2, 2),  # Reduce height and width by 2
        strides=(2, 2),
        padding='valid'  # Reduce size
    ),

    # Second Convolutional Block
    tf.keras.layers.Conv2D(
        filters=8,  # Double the filters for learning mid-level features
        kernel_size=(2, 2),
        strides=(1, 1),
        padding='same',
        activation='relu'
    ),
    tf.keras.layers.MaxPooling2D(
        pool_size=(2, 2),
        strides=(2, 2),
        padding='valid'
    ),

    # # Third Convolutional Block
    # tf.keras.layers.Conv2D(
    #     filters=8,  # Increase filters for high-level features
    #     kernel_size=(3, 3),
    #     strides=(2, 2),
    #     padding='same',
    #     activation='relu'
    # ),
    # tf.keras.layers.MaxPooling2D(
    #     pool_size=(2, 2),
    #     strides=(2, 2),
    #     padding='valid'
    # ),

    # # Fourth Convolutional Block
    
    # tf.keras.layers.Conv2D(
    #     filters=16,  # Increase further for richer high-level features
    #     kernel_size=(3, 3),
    #     strides=(2, 2),
    #     padding='same',
    #     activation='relu'
    # ),
    # tf.keras.layers.MaxPooling2D(
    #     pool_size=(2, 2),
    #     strides=(2, 2),
    #     padding='valid'
    # ),

    # Flatten the features to use later or pass through Dense layers
    tf.keras.layers.Flatten()
])

# Summary of the enhanced model
model.summary()

# Extract features using the enhanced model
features = model.predict(images)

# Print feature shape
print(f"Extracted features shape: {features.shape}")  # Example: (10, 256)


In [ ]:
from sklearn.decomposition import PCA

# Assume features_flat is your flattened feature array from the pretrained model
# For example, features_flat.shape = (2500, 25088) for VGG16

# Initialize PCA to reduce the dimensionality
pca = PCA(n_components=0.95)  # Retain 95% of the variance
features_reduced = pca.fit_transform(features)

print(features_reduced.shape)  

### Tsne


In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [ ]:
tsne = TSNE(n_components=4, random_state=42)
features_2d = tsne.fit_transform(features)

# Create a scatter plot where each class will have a different color
# scatter = plt.scatter(features_2d[:, 0], features_2d[:, 1], c=y, cmap='jet', s=50, alpha=0.7)

# # Add a color bar
# plt.colorbar(scatter, label='Class')

# # Set plot labels and title
# plt.xlabel('t-SNE Component 1')
# plt.ylabel('t-SNE Component 2')
# plt.title('t-SNE visualization of high-dimensional data')

# # Show the plot
# plt.show()

In [ ]:
import umap
umap_model = umap.UMAP(n_components=20, random_state=42)
features_2d = umap_model.fit_transform(features)  # Apply UMAP


In [ ]:
X = features_2d
y = data['targets']

In [ ]:

# Normalize the input data (optional but recommended)

print(X.shape)
print(y.shape)

In [ ]:
print(X[1])

In [ ]:
print(X[0])

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score



# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the input data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define and train the model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
